In [0]:
%sql

SELECT *
FROM 4_prod.pacs_dlt.pacs_examcode_dict
WHERE sct_fsn ILIKE '%dual energy%'

In [0]:
%sql

SELECT *
FROM 4_prod.pacs_dlt.pacs_examcode_dict
WHERE sct_fsn ILIKE '%Chest%'
AND modality_id = 168537006 -- XRay


In [0]:
%sql

SELECT COUNT(DISTINCT PERSON_ID)
FROM 4_prod.raw.mill_person
WHERE DATEDIFF(YEAR, BIRTH_DT_TM, current_timestamp()) >= 50
LIMIT 100

In [0]:
%sql
WITH pt AS (
  SELECT PERSON_ID, BIRTH_DT_TM
  FROM 4_prod.raw.mill_person
  WHERE DATEDIFF(YEAR, BIRTH_DT_TM, current_timestamp()) >= 50
)
SELECT COUNT(DISTINCT PERSONID)
FROM 4_prod.pacs.imaging_metadata AS im
INNER JOIN pt
ON im.PersonId = pt.person_id
WHERE ExamCode IN (
  'XDEXA', 'XVDXA', 'XHDXB', 'XHDXL', 'XHDXR',
  'XLDEX', 'XMDEX', 'XPDEX', 'XBDXW')
AND MillEventDate >= '2024-01-01'
AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50

In [0]:
%sql
WITH pt AS (
  SELECT PERSON_ID, BIRTH_DT_TM
  FROM 4_prod.raw.mill_person
  WHERE DATEDIFF(YEAR, BIRTH_DT_TM, current_timestamp()) >= 50
)
SELECT COUNT(DISTINCT PERSONID)
FROM 4_prod.pacs.imaging_metadata AS im
INNER JOIN pt
ON im.PersonId = pt.person_id
WHERE ExamCode = 'XCHES'
AND MillEventDate >= '2024-01-01'
AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50

In [0]:
%sql

WITH pt AS (
  SELECT PERSON_ID, BIRTH_DT_TM
  FROM 4_prod.raw.mill_person
  WHERE DATEDIFF(YEAR, BIRTH_DT_TM, current_timestamp()) >= 50
),
dxa_pt AS (
  SELECT PERSONID, MillEventDate
  FROM 4_prod.pacs.imaging_metadata AS im
  INNER JOIN pt
  ON im.PersonId = pt.person_id
  WHERE ExamCode IN (
    'XDEXA', 'XVDXA', 'XHDXB', 'XHDXL', 'XHDXR',
    'XLDEX', 'XMDEX', 'XPDEX', 'XBDXW')
  AND MillEventDate >= '2024-01-01'
  AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50
),
xc_pt AS (
  SELECT PERSONID, MillEventDate
  FROM 4_prod.pacs.imaging_metadata AS im
  INNER JOIN pt
  ON im.PersonId = pt.PERSON_ID
  WHERE ExamCode = 'XCHES'
  AND MillEventDate >= '2024-01-01'
  AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50
)
SELECT COUNT(DISTINCT dxa_pt.PERSONID)
FROM dxa_pt
INNER JOIN xc_pt
  ON dxa_pt.PERSONID = xc_pt.PERSONID
  AND ABS(DATEDIFF(DAY, dxa_pt.MillEventDate, xc_pt.MillEventDate)) <= 183

In [0]:
%sql
WITH pt AS (
  SELECT PERSON_ID, BIRTH_DT_TM
  FROM 4_prod.raw.mill_person
  WHERE DATEDIFF(YEAR, BIRTH_DT_TM, current_timestamp()) >= 50
),
dxa_pt AS (
  SELECT PERSONID, MillEventDate
  FROM 4_prod.pacs.imaging_metadata AS im
  INNER JOIN pt
    ON im.PersonId = pt.person_id
  WHERE ExamCode IN (
    'XDEXA', 'XVDXA', 'XHDXB', 'XHDXL', 'XHDXR',
    'XLDEX', 'XMDEX', 'XPDEX', 'XBDXW'
  )
    AND MillEventDate >= '2024-01-01'
    AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50
),
xc_pt AS (
  SELECT PERSONID, MillEventDate
  FROM 4_prod.pacs.imaging_metadata AS im
  INNER JOIN pt
    ON im.PersonId = pt.PERSON_ID
  WHERE ExamCode = 'XCHES'
    AND MillEventDate >= '2024-01-01'
    AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50
)
SELECT 
  dxa_pt.personid, 
  dxa_pt.milleventdate AS dxa_date, 
  xc_pt.milleventdate AS xc_date,
  LEAST(dxa_pt.milleventdate, xc_pt.milleventdate) AS least_date,
  GREATEST(dxa_pt.milleventdate, xc_pt.milleventdate) AS greatest_date
FROM dxa_pt
INNER JOIN xc_pt
  ON dxa_pt.PERSONID = xc_pt.PERSONID
  AND ABS(DATEDIFF(DAY, dxa_pt.MillEventDate, xc_pt.MillEventDate)) <= 183
ORDER BY dxa_pt.personid, dxa_pt.milleventdate ASC

In [0]:
%sql

WITH pt AS (
  SELECT PERSON_ID, BIRTH_DT_TM
  FROM 4_prod.raw.mill_person
  WHERE DATEDIFF(YEAR, BIRTH_DT_TM, current_timestamp()) >= 50
),
dxa_pt AS (
  SELECT PERSONID, MillEventDate
  FROM 4_prod.pacs.imaging_metadata AS im
  INNER JOIN pt
  ON im.PersonId = pt.person_id
  WHERE ExamCode IN (
    'XDEXA', 'XVDXA', 'XHDXB', 'XHDXL', 'XHDXR',
    'XLDEX', 'XMDEX', 'XPDEX', 'XBDXW')
  AND MillEventDate >= '2024-01-01'
  AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50
),
xc_pt AS (
  SELECT PERSONID, MillEventDate
  FROM 4_prod.pacs.imaging_metadata AS im
  INNER JOIN pt
  ON im.PersonId = pt.PERSON_ID
  WHERE ExamCode = 'XCHES'
  AND MillEventDate >= '2024-01-01'
  AND DATEDIFF(YEAR, pt.BIRTH_DT_TM, im.MillEventDate) >= 50
),
joined_pt AS (
  SELECT 
    dxa_pt.personid, 
    LEAST(dxa_pt.milleventdate, xc_pt.milleventdate) AS least_date,
    GREATEST(dxa_pt.milleventdate, xc_pt.milleventdate) AS greatest_date
  FROM dxa_pt
  INNER JOIN xc_pt
    ON dxa_pt.PERSONID = xc_pt.PERSONID
    AND ABS(DATEDIFF(DAY, dxa_pt.MillEventDate, xc_pt.MillEventDate)) <= 183
),
grp_pt AS (
  SELECT personid, MIN(least_date) AS first_date
  FROM joined_pt
  GROUP BY personid
)
SELECT date_trunc('month', first_date) AS month, COUNT(personid) AS person_count
FROM grp_pt
GROUP BY date_trunc('month', first_date)
ORDER BY month



